In [1]:
import pandas as pd
import statistics
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import statistics as stat
from scipy.stats import binom
from sklearn.preprocessing import StandardScaler
from matplotlib.patches import Rectangle
# splitting merge_data into train test and split
from sklearn.model_selection import train_test_split, cross_validate
from scipy import stats
from scipy.stats import normaltest
import os 
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import matplotlib as mpl
import warnings; warnings.filterwarnings(action='once')
# to keep track of training time
import datetime
# logistic regression model
from sklearn.linear_model import LogisticRegression
# metrics used for evaluation
from sklearn.metrics import f1_score, matthews_corrcoef
# visualizations
from yellowbrick.classifier import ClassPredictionError, ConfusionMatrix
from sklearn.metrics import plot_roc_curve
# KNN imputation
from sklearn.impute import KNNImputer
# normalizer
from sklearn.preprocessing import Normalizer
# variance threshold
from sklearn.feature_selection import VarianceThreshold
# RFECV
from sklearn.model_selection import StratifiedKFold
# random forest
from sklearn.ensemble import RandomForestClassifier
# evaluation metric
from sklearn.metrics import matthews_corrcoef, make_scorer
from scipy import stats 
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split 
from sklearn.model_selection import cross_val_score # Evaluate a score by cross-validation
from sklearn.model_selection import GridSearchCV # Exhaustive search over specified parameter values for an estimator
from sklearn.metrics import classification_report # Build a text report showing the main classification metrics
from sklearn.metrics import accuracy_score # Accuracy classification score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay # to show confusion matrix
from sklearn import metrics # Evaluate ROC curve
from sklearn.experimental import enable_iterative_imputer # Enables IterativeImputer. The API and results of this estimator might change without any deprecation cycle
from sklearn.impute import IterativeImputer
from imblearn.combine import SMOTEENN # Combine over- and under-sampling using SMOTE and Edited Nearest Neighbours
from sklearn import decomposition
from boruta import BorutaPy
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.metrics import plot_confusion_matrix
from matplotlib.colors import ListedColormap

Read files 

In [2]:
def read_secom():
    path ='../secom.data'
    df = pd.read_csv(path, delimiter=' ', header=None, na_values=['NaN'])
    df.columns = ['feature'+str(x+1) for x in range(len(df.columns))]
    return df

#%%
def read_labels():
    path = '../secom_labels.data'
    df = pd.read_csv(path, delimiter=' ', header=None, na_values=['NaN'])
    df.columns = ['status','timestamp']
    df['timestamp'] = pd.to_datetime(df['timestamp'],dayfirst=True)
    return df

#read 2 df 
df_features = read_secom()
df_target = read_labels()

# Data Preparation

## Merge data

In [3]:
df= pd.concat([df_features,df_target],axis=1)

## Split Training and test data 

In [4]:
x = df.iloc[:,:590]
y = df.iloc[:,590]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1,stratify=y)

In [5]:
x_train.isna().sum().sum()

33219

## reducing dimensionality 

### remove missing values 

In [6]:
def null_values(df):
    """a function to show null values with percentage"""
    nv=pd.concat([df.isnull().sum(), 100 * df.isnull().sum()/df.shape[0]],axis=1).rename(columns={0:'Missing_Records', 1:'Percentage (%)'})
    return nv[nv.Missing_Records>0].sort_values('Missing_Records', ascending=False)

In [8]:
x_train_na = null_values(x_train)
x_train_na

,Missing_Records,Percentage (%)
feature293,1144,91.300878
feature158,1144,91.300878
feature159,1144,91.300878
feature294,1144,91.300878
feature221,1072,85.554669
...,...,...
feature500,1,0.079808
feature367,1,0.079808
feature377,1,0.079808
feature378,1,0.079808


In [9]:
miss_threshold=80
x_train_na= x_train_na[x_train_na["Percentage (%)"] > miss_threshold]
x_train_na

,Missing_Records,Percentage (%)
feature293,1144,91.300878
feature158,1144,91.300878
feature159,1144,91.300878
feature294,1144,91.300878
feature221,1072,85.554669
feature86,1072,85.554669
feature359,1072,85.554669
feature493,1072,85.554669


In [10]:
x_train_new = x_train.drop(axis=1, columns=x_train_na.index)
x_train_new.shape

(1253, 582)

### removing constant voltality

In [11]:
x_train_new_eda= x_train_new.describe().T
x_train_new_eda

,count,mean,std,min,25%,50%,75%,max
feature1,1248.0,3014.113686,73.877303,2743.2400,2966.2300,3011.40500,3056.310000,3356.3500
feature2,1247.0,2496.400585,79.188651,2162.8700,2452.3350,2500.38000,2539.600000,2846.4400
feature3,1243.0,2200.216525,29.939025,2060.6600,2180.8611,2200.98890,2218.055500,2315.2667
feature4,1243.0,1394.907987,436.632421,0.0000,1083.8858,1283.43680,1593.122000,3715.0417
feature5,1243.0,4.018582,54.572445,0.6815,1.0160,1.31710,1.525700,1114.5366
...,...,...,...,...,...,...,...,...
feature586,1252.0,3.084501,3.968128,1.1975,2.3058,2.74655,3.258475,99.3032
feature587,1252.0,0.021521,0.012421,-0.0060,0.0134,0.02070,0.027600,0.1028
feature588,1252.0,0.016344,0.008698,0.0042,0.0106,0.01480,0.019800,0.0799
feature589,1252.0,0.005248,0.002850,0.0012,0.0033,0.00460,0.006300,0.0286


In [12]:
x_train_new_eda[x_train_new_eda['std']==0]
x_train_new_std= x_train_new_eda[x_train_new_eda["std"] == 0]
x_train_new_std

,count,mean,std,min,25%,50%,75%,max
feature6,1243.0,100.0,0.0,100.0,100.0,100.0,100.0,100.0
feature14,1251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
feature43,1253.0,70.0,0.0,70.0,70.0,70.0,70.0,70.0
feature50,1253.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
feature53,1253.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
feature535,1247.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
feature536,1247.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
feature537,1247.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
feature538,1247.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
x_train_new_2 = x_train_new.drop(axis=1, columns=x_train_new_std.index)
x_train_new_2.shape

(1253, 466)

## Outlier Treatment

In [14]:
x_train_3s=x_train_new_2.copy()

In [15]:
def zscore(df,cols,thr):
    for col in cols:
        mean=df[col].mean()
        std=df[col].std()
        upper_bound=mean+ thr*std
        lower_bound=mean- thr*std
        df[col]=np.where(df[col]>upper_bound,upper_bound,np.where(df[col]<lower_bound,lower_bound,df[col]))

In [16]:
zscore(x_train_3s,x_train_3s.columns,3)

# Checking

In [49]:
x_train_new_2['feature4'].mean()

1394.907986725665

In [48]:

x_train_new_2['feature4'].std()

436.63242083048897

In [50]:
x_train_new_2['feature4'].mean()+3*x_train_new_2['feature4'].std()

2704.805249217132

In [51]:
x_train_new_2['feature4'].max()

3715.0417

In [52]:
x_train_3s['feature4'].max()


2704.805249217132

## Imputation

### KNN Imputation

In [17]:
# scaling before distanced based algo
scaler = MinMaxScaler()
x_train_scaled = pd.DataFrame(scaler.fit_transform(x_train_3s), columns=x_train_3s.columns)
x_train_scaled

,feature1,feature2,feature3,feature4,feature5,feature7,feature8,feature9,feature10,feature11,...,feature581,feature582,feature583,feature584,feature585,feature586,feature587,feature588,feature589,feature590
0,0.648120,0.201366,0.538010,0.452172,0.006439,0.392017,0.787343,0.527160,0.680406,0.663699,...,0.345437,0.190010,0.514663,0.075172,0.110693,0.071958,0.527907,0.397495,0.373085,0.182951
1,0.471228,0.489118,0.250017,0.318042,0.003555,0.606695,0.806973,0.430272,0.451391,0.514345,...,NaN,NaN,0.224461,0.208979,0.166039,0.205512,0.353482,0.611932,0.547721,0.436456
2,0.564671,0.250573,0.351025,0.514797,0.002089,0.753605,0.865862,0.613416,0.734617,0.735677,...,NaN,NaN,0.319527,0.081186,0.110693,0.079049,0.413682,0.640698,0.627101,0.368023
3,0.510211,0.300896,0.632400,0.543962,0.005265,0.467181,0.715366,0.483910,0.387223,0.678095,...,0.191093,0.137507,0.744823,0.061641,0.089938,0.057659,0.530994,0.258894,0.206388,0.132398
4,0.513934,0.368646,0.597947,0.633548,0.001678,0.476510,0.777528,0.610033,0.401606,0.399181,...,NaN,NaN,0.559694,0.130800,0.124529,0.125897,0.347307,0.188287,0.246077,0.183597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248,0.597451,0.468702,0.581926,0.602601,0.002028,0.483641,0.872406,0.581040,0.146039,0.582724,...,NaN,NaN,0.694788,0.172896,0.145284,0.165038,0.498579,0.196132,0.166698,0.118079
1249,0.490065,0.645495,0.636173,0.467074,0.004245,0.502029,0.744811,0.486810,0.421520,0.359593,...,NaN,NaN,0.404586,0.027062,0.041510,0.027184,0.517102,0.452412,0.508031,0.207902
1250,0.545630,0.784278,0.382138,0.741741,0.005230,0.349415,0.829874,0.448393,0.563133,0.661900,...,NaN,NaN,0.349548,0.102234,0.117611,0.100454,0.552604,0.193517,0.214325,0.103733
1251,0.525033,0.514521,0.385973,1.000000,0.004806,0.000000,0.875677,0.304874,0.431477,0.436969,...,0.029399,0.127442,0.604725,0.147337,0.172957,0.141052,0.277846,0.033996,0.031752,0.122707


In [18]:
# impute missing values and save it as a temporary dataset.
knn = KNNImputer()
knn.fit(x_train_scaled)
imputed_train = pd.DataFrame(knn.transform(x_train_scaled), columns = x_train_scaled.columns)

In [24]:
imputed_train

,feature1,feature2,feature3,feature4,feature5,feature7,feature8,feature9,feature10,feature11,...,feature581,feature582,feature583,feature584,feature585,feature586,feature587,feature588,feature589,feature590
0,0.648120,0.201366,0.538010,0.452172,0.006439,0.392017,0.787343,0.527160,0.680406,0.663699,...,0.345437,0.190010,0.514663,0.075172,0.110693,0.071958,0.527907,0.397495,0.373085,0.182951
1,0.471228,0.489118,0.250017,0.318042,0.003555,0.606695,0.806973,0.430272,0.451391,0.514345,...,0.423344,0.230365,0.224461,0.208979,0.166039,0.205512,0.353482,0.611932,0.547721,0.436456
2,0.564671,0.250573,0.351025,0.514797,0.002089,0.753605,0.865862,0.613416,0.734617,0.735677,...,0.277819,0.172041,0.319527,0.081186,0.110693,0.079049,0.413682,0.640698,0.627101,0.368023
3,0.510211,0.300896,0.632400,0.543962,0.005265,0.467181,0.715366,0.483910,0.387223,0.678095,...,0.191093,0.137507,0.744823,0.061641,0.089938,0.057659,0.530994,0.258894,0.206388,0.132398
4,0.513934,0.368646,0.597947,0.633548,0.001678,0.476510,0.777528,0.610033,0.401606,0.399181,...,0.236661,0.336388,0.559694,0.130800,0.124529,0.125897,0.347307,0.188287,0.246077,0.183597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248,0.597451,0.468702,0.581926,0.602601,0.002028,0.483641,0.872406,0.581040,0.146039,0.582724,...,0.217552,0.130255,0.694788,0.172896,0.145284,0.165038,0.498579,0.196132,0.166698,0.118079
1249,0.490065,0.645495,0.636173,0.467074,0.004245,0.502029,0.744811,0.486810,0.421520,0.359593,...,0.249890,0.155535,0.404586,0.027062,0.041510,0.027184,0.517102,0.452412,0.508031,0.207902
1250,0.545630,0.784278,0.382138,0.741741,0.005230,0.349415,0.829874,0.448393,0.563133,0.661900,...,0.318978,0.106352,0.349548,0.102234,0.117611,0.100454,0.552604,0.193517,0.214325,0.103733
1251,0.525033,0.514521,0.385973,1.000000,0.004806,0.000000,0.875677,0.304874,0.431477,0.436969,...,0.029399,0.127442,0.604725,0.147337,0.172957,0.141052,0.277846,0.033996,0.031752,0.122707


# Checking

In [58]:
x_train_scaled.isna().sum().sum()


23795

In [19]:
imputed_train.isna().sum().sum()

0

## Feature Selection

### BORUTA

In [25]:
#Boruta function

def BorutaFeatureSelection (X, y) :
    feature_names = np.array(X.columns)

    # define random forest classifier
    model = RandomForestClassifier(n_jobs=-1, class_weight='balanced_subsample', max_depth=5, random_state=100)
    model.fit(X, y)
    
    # define Boruta feature selection method
    feature_selector = BorutaPy(model, n_estimators='auto', verbose=2, random_state=100, max_iter=200)

    # find all relevant features
    feature_selector.fit(X.to_numpy(),y)

    # check selected features
    feature_selector.support_

    # check ranking of features
    feature_selector.ranking_

    # zip feature names, ranks, and decisions 
    feature_ranks = list(zip(feature_names, 
                             feature_selector.ranking_, 
                             feature_selector.support_))

    # print the results
    for feat in feature_ranks:
        print('Feature: {:<30} Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))
        
    final_features = list()
    indexes = np.where(feature_selector.ranking_ <= 1)
    for x in np.nditer(indexes):
        final_features.append(feature_names[x])
    print(final_features)
    
 # call transform() on X to filter it down to selected features
    return pd.DataFrame(X.filter(final_features)) , final_features

In [26]:
# apply Boruta and store selected features in a variable
X_train , final_features = BorutaFeatureSelection(imputed_train,y_train)

c:\Users\hinat\anaconda3\lib\site-packages\boruta\boruta_py.py:260: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

c:\Users\hinat\anaconda3\lib\site-packages\boruta\boruta_py.py:265: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecat

Iteration: 	1 / 200
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	2 / 200
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	3 / 200
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	4 / 200
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	5 / 200
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	6 / 200
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	7 / 200
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	8 / 200
Confirmed: 	0
Tentative: 	16
Rejected: 	450
Iteration: 	9 / 200
Confirmed: 	1
Tentative: 	15
Rejected: 	450
Iteration: 	10 / 200
Confirmed: 	1
Tentative: 	15
Rejected: 	450
Iteration: 	11 / 200
Confirmed: 	1
Tentative: 	15
Rejected: 	450
Iteration: 	12 / 200
Confirmed: 	3
Tentative: 	13
Rejected: 	450
Iteration: 	13 / 200
Confirmed: 	3
Tentative: 	13
Rejected: 	450
Iteration: 	14 / 200
Confirmed: 	3
Tentative: 	13
Rejected: 	450
Iteration: 	15 / 200
Confirmed: 	3
Tentative: 	13
Rejected: 	450
Iteration: 	16 / 200
Confirmed: 	4
Tentat

c:\Users\hinat\anaconda3\lib\site-packages\boruta\boruta_py.py:319: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

c:\Users\hinat\anaconda3\lib\site-packages\boruta\boruta_py.py:325: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations




BorutaPy finished running.

Iteration: 	200 / 200
Confirmed: 	14
Tentative: 	2
Rejected: 	450
Feature: feature1                       Rank: 3,  Keep: False
Feature: feature2                       Rank: 358,  Keep: False
Feature: feature3                       Rank: 260,  Keep: False
Feature: feature4                       Rank: 132,  Keep: False
Feature: feature5                       Rank: 436,  Keep: False
Feature: feature7                       Rank: 281,  Keep: False
Feature: feature8                       Rank: 227,  Keep: False
Feature: feature9                       Rank: 71,  Keep: False
Feature: feature10                      Rank: 352,  Keep: False
Feature: feature11                      Rank: 155,  Keep: False
Feature: feature12                      Rank: 102,  Keep: False
Feature: feature13                      Rank: 302,  Keep: False
Feature: feature15                      Rank: 104,  Keep: False
Feature: feature16                      Rank: 262,  Keep: False
Feature: fe

In [28]:
X_train

,feature20,feature22,feature34,feature60,feature122,feature130,feature131,feature206,feature248,feature342,feature349,feature427,feature478,feature520
0,0.442228,0.700874,0.193347,0.459091,0.457841,0.605900,0.825924,0.167284,0.058279,0.159031,0.294321,0.271795,0.120407,0.053742
1,0.556530,0.508093,0.123900,0.422373,0.457841,0.651403,0.848418,0.121112,0.119088,0.116549,0.371196,0.084510,0.086977,0.109580
2,0.609865,0.202565,0.202283,0.624540,0.265462,0.560398,0.119139,0.157131,0.132277,0.147298,0.131786,0.335955,0.124078,0.121430
3,0.512533,0.401699,0.191819,0.617883,0.405374,0.613484,0.786303,0.132232,1.000000,0.142078,0.432696,0.365039,0.093620,1.000000
4,0.525912,0.072950,0.175154,0.464895,0.702687,0.674170,0.674598,0.211765,0.066024,0.205703,0.307500,0.124838,0.160813,0.060915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248,0.544409,0.204026,0.268692,0.504152,0.807621,0.454209,0.653126,0.109508,0.071621,0.101315,0.353625,0.164139,0.081905,0.066158
1249,0.585353,0.490714,0.146498,0.499608,0.510308,0.537630,0.573884,0.175987,0.077833,0.163143,0.175714,0.055341,0.138371,0.071380
1250,0.674962,0.343938,0.162079,0.646226,0.772643,0.696922,0.871168,0.179855,0.530260,0.200291,0.362410,0.418581,0.130652,0.482659
1251,0.593703,0.413310,0.259703,0.406870,0.335418,0.605900,0.203493,0.102981,0.080057,0.108951,0.297835,0.298025,0.073836,0.074285


In [61]:
# prepare the data for boruta by fitting it to random forest Classifier 
from sklearn.ensemble import RandomForestClassifier

# define random forest classifier
forest = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
forest.fit(imputed_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=5, n_jobs=-1)

In [62]:
iteration=200
# define Boruta feature selection method
feat_selector = BorutaPy(forest, n_estimators='auto', verbose=2, random_state=1,max_iter=iteration)

# find all relevant features
feat_selector.fit(np.array(imputed_train),np.array(y_train))

c:\Users\hinat\anaconda3\lib\site-packages\boruta\boruta_py.py:260: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

c:\Users\hinat\anaconda3\lib\site-packages\boruta\boruta_py.py:265: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecat

Iteration: 	1 / 200
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	2 / 200
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	3 / 200
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	4 / 200
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	5 / 200
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	6 / 200
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	7 / 200
Confirmed: 	0
Tentative: 	466
Rejected: 	0
Iteration: 	8 / 200
Confirmed: 	0
Tentative: 	18
Rejected: 	448
Iteration: 	9 / 200
Confirmed: 	2
Tentative: 	16
Rejected: 	448
Iteration: 	10 / 200
Confirmed: 	2
Tentative: 	16
Rejected: 	448
Iteration: 	11 / 200
Confirmed: 	2
Tentative: 	16
Rejected: 	448
Iteration: 	12 / 200
Confirmed: 	5
Tentative: 	13
Rejected: 	448
Iteration: 	13 / 200
Confirmed: 	5
Tentative: 	13
Rejected: 	448
Iteration: 	14 / 200
Confirmed: 	5
Tentative: 	13
Rejected: 	448
Iteration: 	15 / 200
Confirmed: 	5
Tentative: 	13
Rejected: 	448
Iteration: 	16 / 200
Confirmed: 	6
Tentat

c:\Users\hinat\anaconda3\lib\site-packages\boruta\boruta_py.py:319: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

c:\Users\hinat\anaconda3\lib\site-packages\boruta\boruta_py.py:325: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


BorutaPy(estimator=RandomForestClassifier(class_weight='balanced', max_depth=5,
                                          n_estimators=113, n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x2ACD0D6CD40),
         max_iter=200, n_estimators='auto',
         random_state=RandomState(MT19937) at 0x2ACD0D6CD40, verbose=2)

In [63]:
print("Ranking: ",feat_selector.ranking_)          
print("No. of significant features: ", feat_selector.n_features_) 

Ranking:  [  1 436 144 259 314 384 216  50 188 188  71 210  76 291  60  84 425   1
 337  10 255 317 155 145 148 152  12 360 170  10 130   1 388 151 206 127
 182 282  46  50 261 371 200 289 188 223 395 146 402 367 150  79 377  54
   1 335 143 159  93   1  20 148 270 184 143 264 268 167 450 332  41 343
  23 154 244 127 366 122 418 239 411 329 284  14  68 301 295 442 440 343
 239 315 358 413  69   2 423 251 407 221 334  38 220   5  76 287 446  12
 304   8 408 192  93   1  67  64   4  30   4  65 239   1   1 361 111  17
 415 197 291 166 324  73 188 348  83 208 306  85  44 405 172 371  15  27
 393 229 138 181  58  52 326  37 164 418 435 439 400 211 249 307 172 296
 132 205 406 292  32 252 140  25 112 374 157  52  82 319 114 223 253  89
 139 271  79 337   1 450 374 363 450  34 269 304  97 118 199 168 384 104
 214 405 284 377 428 329  74 242 280 175  99 239  21   1 278 444 333 377
 398 412 236 324  80 163 348 262 397 298 305 125 348 429 120 202 137  87
  29 282 162 326  18  27 363 115 132 257 

In [64]:
feat_selector.support_

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False,  True,  True, False, False, False,
       False, False,

In [65]:
green_area = imputed_train.columns[feat_selector.support_].to_list()
blue_area = imputed_train.columns[feat_selector.ranking_].to_list()

In [66]:
X_best_features= feat_selector.transform(imputed_train.to_numpy())